In [ ]:
## First we will visualize the train data

In [1]:
import numpy as np
import pandas as pd
import sklearn

In [18]:
!ls -l china-real-estate-demand-prediction/train

total 4088
-rw-rw-r--@ 1 deepak  staff    6127  8 Aug 07:56 city_indexes.csv
-rw-rw-r--@ 1 deepak  staff  130589  8 Aug 07:56 city_search_index.csv
-rw-rw-r--@ 1 deepak  staff  174046  8 Aug 07:56 land_transactions_nearby_sectors.csv
-rw-rw-r--@ 1 deepak  staff  168460  8 Aug 07:56 land_transactions.csv
-rw-rw-r--@ 1 deepak  staff  590991  8 Aug 07:56 new_house_transactions_nearby_sectors.csv
-rw-rw-r--@ 1 deepak  staff  376737  8 Aug 07:56 new_house_transactions.csv
-rw-rw-r--@ 1 deepak  staff  322116  8 Aug 07:56 pre_owned_house_transactions_nearby_sectors.csv
-rw-rw-r--@ 1 deepak  staff  241849  8 Aug 07:56 pre_owned_house_transactions.csv
-rw-rw-r--@ 1 deepak  staff   63502  8 Aug 07:56 sector_POI.csv


In [48]:
city_indexes = pd.read_csv('china-real-estate-demand-prediction/train/city_indexes.csv')
city_search_index = pd.read_csv('china-real-estate-demand-prediction/train/city_search_index.csv')

land_transactions = pd.read_csv('china-real-estate-demand-prediction/train/land_transactions.csv')
land_transactions_nearby_sectors = pd.read_csv('china-real-estate-demand-prediction/train/land_transactions_nearby_sectors.csv')

new_house_transactions = pd.read_csv('china-real-estate-demand-prediction/train/new_house_transactions.csv')
new_house_transactions_nearby_sectors = pd.read_csv('china-real-estate-demand-prediction/train/new_house_transactions_nearby_sectors.csv')

pre_owned_house_transactions = pd.read_csv('china-real-estate-demand-prediction/train/pre_owned_house_transactions.csv')
pre_owned_house_transactions_nearby_sectors = pd.read_csv('china-real-estate-demand-prediction/train/pre_owned_house_transactions_nearby_sectors.csv')

sector_POI = pd.read_csv('china-real-estate-demand-prediction/train/sector_POI.csv')

#city_indexes.head(15)
#land_transactions.head()
#land_transactions_nearby_sectors.head()
#new_house_transactions.head()
#new_house_transactions_nearby_sectors.shape #head()
#pre_owned_house_transactions.head()
#pre_owned_house_transactions_nearby_sectors.head()
sector_POI.head()

,sector,sector_coverage,population_scale,residential_area,office_building,commercial_area,resident_population,office_population,number_of_shops,catering,...,medical_health_rehabilitation_institution_dense,medical_health_first_aid_center_dense,medical_health_blood_donation_station_dense,medical_health_disease_prevention_institution_dense,medical_health_general_hospital_dense,medical_health_clinic_dense,education_training_school_education_middle_school_dense,education_training_school_education_primary_school_dense,education_training_school_education_kindergarten_dense,education_training_school_education_research_institution_dense
0,sector 23,0.410668,113800,68,58,4,88000,36000,2398,597,...,0.000024,0.000000e+00,0.0,0.000000e+00,0.000007,2.550000e-05,1.220000e-06,6.080000e-06,0.000013,3.650000e-06
1,sector 80,0.426227,74388,23,9,0,37173,53683,837,277,...,0.000001,0.000000e+00,0.0,1.910000e-07,0.000001,5.740000e-07,3.830000e-07,3.830000e-07,0.000002,2.490000e-06
2,sector 84,0.448303,146800,136,62,1,118000,40000,2776,697,...,0.000015,7.480000e-07,0.0,7.480000e-07,0.000006,1.270000e-05,5.230000e-06,4.490000e-06,0.000010,5.230000e-06
3,sector 16,0.688514,1435300,2542,434,31,1023000,589000,62837,11222,...,0.000149,0.000000e+00,0.0,7.470000e-05,0.000100,3.609820e-04,2.490000e-05,4.980000e-05,0.000324,1.244770e-04
4,sector 27,0.366972,40600,24,22,1,22000,27000,805,226,...,0.000005,0.000000e+00,0.0,0.000000e+00,0.000004,9.150000e-07,0.000000e+00,9.150000e-07,0.000005,9.150000e-07
